In [8]:
import numpy as np
from ldpc.codes import ring_code
from ldpc.code_util import compute_code_distance
import ldpc.mod2 as mod2
from ldpc import protograph as pt
from ldpc.codes import hamming_code
from bposd.css import css_code
from bposd.hgp import hgp
from lifted_hgp import *
from css_ss_decode_sim import css_ss_decode_sim
#from bposd.css_decode_sim import css_decode_sim

In [37]:
a1=pt.array([
        [(0), (11), (7), (12)],
        [(1), (8), (1), (8)],
        [(11), (0), (4), (8)],
        [(6), (2), (4), (12)]])

qcode = lifted_hgp(lift_parameter=13,a=a1,b=a1)

sim_input={
"error_rate": 0.10, #the physical error rate on the qubits
"target_runs": 1000, #the number of cycles to simulate
"bp_method": "minimum_sum", #the bp method
"ms_scaling_factor": 0.625, # the min-sum scaling factor
"osd_method": "osd_e", # OSD method
"osd_order": 10, #OSD order
"xyz_error_bias": [10,1,1], #the relative XYZ bias
"hadamard_rotate": True, #Hadamard rotate
"hadamard_rotate_sector1_length": qcode.hx1.shape[1], #the length of sector 1 qubit block. All qubits in sector 2 are Hadamard rotated
"channel_update": "x->z", # the channel update orientation
'max_iter': int(qcode.N/10), #the interation depth for BP
'tqdm_disable': False #show live stas
}
css_decode_sim(hx=qcode.hx,hz=qcode.hz,**sim_input)

RNG Seed: 2971585989
Constructing CSS code from hx and hz matrices...
Checking the CSS code is valid...
<Unnamed CSS code>
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params [416,18,nan]


d_max: 26; OSDW_WER: 0.0614±0.018%; OSDW: 1.1±0.33%; OSD0: 2.2±0.46%;: 100% 1000/1000 [00:31<00:00, 31.69it/s] 
